In [1]:
import numpy as np
from datetime import datetime


from CaliPytion import Calibrator, Standard

## Generate artificial data for a `Standard`

In [2]:
def random(xs):
    return xs+np.random.normal(0, 0.03, 1)


SLOPE = 0.014
CONC = np.linspace(0, 200, 11)
ABSO = CONC*SLOPE

In [3]:
# Generate standard
standard = Standard(
    species_id="s0",
    name="ABTS",
    wavelength=340,
    ph=2,
    temperature=25,
    temperature_unit="C",
    created=datetime.now(),
)

# Add samples with noise
for conc, abso in zip(CONC, ABSO):
    standard.add_to_samples(
        concentration=conc,
        conc_unit="umol / l",
        signal=random(abso),
    )
    standard.add_to_samples(
        concentration=conc,
        conc_unit="umol / l",
        signal=random(abso),
    )
    standard.add_to_samples(
        concentration=conc,
        conc_unit="umol / l",
        signal=random(abso),
    )

## Initialize a `Calibrator` from a `Standard`

By defining a cutoff value, samples with a higher signal than the defined cutoff are ignored for the calibration.
For concentration calculation, the model does not extrapolate beyond the cutoff value by default.

In [4]:
# initialize calibrator
calibrator = Calibrator.from_standard(standard, cutoff=2.5)

CaliPytion contains predefined models, which can be used for calibration.
Alternatively, custom models can be added. 

```{note}
The equation of a model must include 'signal' and 'concentration' as variables.
```

In [5]:
exponential = calibrator.add_to_models(
    name="exponential",
    equation="a * exp(b * concentration) = signal",
)

By calling `fit_models`, all models are fitted to the data defined in `Standard`. A report, summarizing statistical parameters of each model is generated.

In [6]:
# Fit all defined models
calibrator.fit_models()

,AIC,R squared,RMSD
Model Name,,,
linear,-198,0.9988,0.0248
quadratic,-198,0.9989,0.0236
rational,-198,0.9989,0.0236
cubic,-197,0.9990,0.0233
exponential,-86,0.9307,0.1899


## Model visualization

Models can be visualized, displaying the measured samples used for fitting of thee model, as well as the fitted model. Additionally, the residuals of the model are visualized.

In [7]:
# Get the linear, predefined model
linear = [model for model in calibrator.models if model.name == "linear"][0]

# Visualize the models
calibrator.visualize(linear)
calibrator.visualize(exponential)

## Save model to `Standard`

After assessing different models, the best model can be saved to the `Standard` object.

In [8]:
standard = calibrator.save_fitted_model(linear)

## Calculate concentrations

Models can be used to calculate concentrations of unknown signals. 

In [9]:
linear.calculate([1, 0.1, 2.234])

[71.03578583517627, 7.103578583517627, 158.69394555578378]

If the signal is out of calibration bonds, the calculate method returns `float('nan')` values for the respective signals.

In [10]:
linear.calculate([1, 100, 5, 0.5])

[71.03578583517627, nan, nan, 35.517892917588135]